In [1]:
!pip install python-docx
!pip install -q -U google-generativeai
!pip install sentence-transformers
!pip install faiss-cpu
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 46.0 MB/s eta 0:00:00


In [12]:
import pandas as pd
import random
import numpy as np
import os
import re
import json
import google.generativeai as genai
os.environ['GOOGLE_API_KEY'] = "Your google key"

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model_google = genai.GenerativeModel('gemini-1.5-flash')

In [3]:
import docx
from docx import Document

In [4]:
# Load document that the RAG with extract style and them recreate

import pdfplumber

def extract_text_from_pdf(pdf_path):
    full_text = ""

    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n\n"

    return full_text

# Usage
pdf_path = "/content/McDonalds_EmployeeHandbook.pdf"
full_text = extract_text_from_pdf(pdf_path)
print(full_text)

Crew Member Handbook
January, 2015
AT-WILL EMPLOYMENT STATEMENT
These policies are intended to provide you with general information about STC, ENCE, NAV, and JENESIS,
Inc. (also referred to as the “Company”) employment policies, practices, guidelines and benefits. They
are not intended to create legally enforceable obligations on the part of the Company or its employees.
The policies herein supersede and replace all previous personnel policies, practices, and guidelines that
are inconsistent with any of the provisions contained herein. The Company reserves full discretion to add
to, modify or delete provision of this Employee Handbook at any time and without advance notice. Any
words or phrases used in this Employee handbook shall be interpreted by the Company in its sole and
exclusive discretion.
THIS HANDBOOK IS NOT INTENDED TO CREATE EITHER AN EXPRESS OR AN IMPLIED CONTRACT OF
EMPLOYMENT. RATHER, IT IS INTENDED TO DESCRIBE THE COMPANY AND ITS PRESENT POLICIES
AND PROCEDURES. YOUR EM

In [5]:
prmomp_preprocess ="please extract the sections from the employee handbook so that its has all sections eg - statements , policies and all other section names"

In [6]:
response_sections =model_google.generate_content("find the section headers from this text" +  prmomp_preprocess + full_text)
response_sections.text

'Here are the section headers from the provided text:\n\n**Main Sections:**\n\n* **AT-WILL EMPLOYMENT STATEMENT**\n* **OPEN DOOR POLICY**\n* **DISCIPLINARY STATEMENT**\n* **GROOMING POLICY**\n* **WORK SCHEDULE PROCEDURES**\n* **NON-DISCRIMINATION & HARASSMENT POLICY**\n* **SECURITY PROCEDURES**\n* **CASH REGISTER SHORTAGES AND PROCEDURES**\n* **BREAK POLICY**\n* **SMOKE POLICY**\n* **EMPLOYEE MEAL POLICY**\n* **WORKPLACE BEHAVIOR**\n* **SPECIAL STORE RULES**\n* **MISUSE OF COMPANY PRODUCT AND/OR EQUIPMENT**\n* **SOLICITATION AND DISTRIBUTION POLICY**\n* **CRIMINAL SEXUAL OFFENSE**\n* **ONLINE COMMUNICATIONS POLICY**\n* **EMPLOYEE RIGHTS AND RESPONSIBILITIES UNDER THE FAMILY AND MEDICAL LEAVE ACT**\n* **NEW HEALTH INSURANCE MARKETPLACE COVERAGE OPTIONS AND YOUR HEALTH COVERAGE**\n\n**Subsections:**\n\n* **EEO Policy**\n* **Americans With Disabilities Act**\n* **Non-Discrimination & Harassment Policy**\n* **Unlawful Harassment**\n* **No Retaliation**\n* **Complaint Procedure**\n* **Zero 

In [7]:
# make the prompt in which mention the keys of the sections of the document
prompt = "please organise the employee handbook in the following sections"+ response_sections.text + " Important to Keep the sentences \
as they are from the document dont change original text and style. If a section is missing, leave it blank."

In [8]:
response =model_google.generate_content("Organize the employee handbook into specific sections.\
 the output should be a simple list of section names , no 2nd level lists so i can use this in a python program , clean list" +  prompt + full_text)
response.text

'Here\'s a list of section names from the provided text, organized into a simple list for use in a Python program:\n\n```python\nsections = [\n    "AT-WILL EMPLOYMENT STATEMENT",\n    "OPEN DOOR POLICY",\n    "DISCIPLINARY STATEMENT",\n    "GROOMING POLICY",\n    "WORK SCHEDULE PROCEDURES",\n    "NON-DISCRIMINATION & HARASSMENT POLICY",\n    "SECURITY PROCEDURES",\n    "CASH REGISTER SHORTAGES AND PROCEDURES",\n    "BREAK POLICY",\n    "SMOKE POLICY",\n    "EMPLOYEE MEAL POLICY",\n    "WORKPLACE BEHAVIOR",\n    "SPECIAL STORE RULES",\n    "MISUSE OF COMPANY PRODUCT AND/OR EQUIPMENT",\n    "SOLICITATION AND DISTRIBUTION POLICY",\n    "CRIMINAL SEXUAL OFFENSE",\n    "ONLINE COMMUNICATIONS POLICY",\n    "EMPLOYEE RIGHTS AND RESPONSIBILITIES UNDER THE FAMILY AND MEDICAL LEAVE ACT",\n    "NEW HEALTH INSURANCE MARKETPLACE COVERAGE OPTIONS AND YOUR HEALTH COVERAGE",\n    "EEO Policy",\n    "Americans With Disabilities Act",\n    "Non-Discrimination & Harassment Policy",\n    "Unlawful Harassm

In [9]:
len(response.text)

1729

In [10]:
employee_section_list= response_sections.text.split("\n")
employee_section_list

['Here are the section headers from the provided text:',
 '',
 '**Main Sections:**',
 '',
 '* **AT-WILL EMPLOYMENT STATEMENT**',
 '* **OPEN DOOR POLICY**',
 '* **DISCIPLINARY STATEMENT**',
 '* **GROOMING POLICY**',
 '* **WORK SCHEDULE PROCEDURES**',
 '* **NON-DISCRIMINATION & HARASSMENT POLICY**',
 '* **SECURITY PROCEDURES**',
 '* **CASH REGISTER SHORTAGES AND PROCEDURES**',
 '* **BREAK POLICY**',
 '* **SMOKE POLICY**',
 '* **EMPLOYEE MEAL POLICY**',
 '* **WORKPLACE BEHAVIOR**',
 '* **SPECIAL STORE RULES**',
 '* **MISUSE OF COMPANY PRODUCT AND/OR EQUIPMENT**',
 '* **SOLICITATION AND DISTRIBUTION POLICY**',
 '* **CRIMINAL SEXUAL OFFENSE**',
 '* **ONLINE COMMUNICATIONS POLICY**',
 '* **EMPLOYEE RIGHTS AND RESPONSIBILITIES UNDER THE FAMILY AND MEDICAL LEAVE ACT**',
 '* **NEW HEALTH INSURANCE MARKETPLACE COVERAGE OPTIONS AND YOUR HEALTH COVERAGE**',
 '',
 '**Subsections:**',
 '',
 '* **EEO Policy**',
 '* **Americans With Disabilities Act**',
 '* **Non-Discrimination & Harassment Policy**',

In [13]:

# Cleaned list comprehension
cleaned_sections = [
    re.sub(r'^\d+\.\s|\*\*|\s*\*\s*', '', section).strip()  # Remove leading numbers, asterisks, and strip spaces
    for section in employee_section_list
    if section.strip()  # Ignore empty strings
]

print(cleaned_sections)

['Here are the section headers from the provided text:', 'Main Sections:', 'AT-WILL EMPLOYMENT STATEMENT', 'OPEN DOOR POLICY', 'DISCIPLINARY STATEMENT', 'GROOMING POLICY', 'WORK SCHEDULE PROCEDURES', 'NON-DISCRIMINATION & HARASSMENT POLICY', 'SECURITY PROCEDURES', 'CASH REGISTER SHORTAGES AND PROCEDURES', 'BREAK POLICY', 'SMOKE POLICY', 'EMPLOYEE MEAL POLICY', 'WORKPLACE BEHAVIOR', 'SPECIAL STORE RULES', 'MISUSE OF COMPANY PRODUCT AND/OR EQUIPMENT', 'SOLICITATION AND DISTRIBUTION POLICY', 'CRIMINAL SEXUAL OFFENSE', 'ONLINE COMMUNICATIONS POLICY', 'EMPLOYEE RIGHTS AND RESPONSIBILITIES UNDER THE FAMILY AND MEDICAL LEAVE ACT', 'NEW HEALTH INSURANCE MARKETPLACE COVERAGE OPTIONS AND YOUR HEALTH COVERAGE', 'Subsections:', 'EEO Policy', 'Americans With Disabilities Act', 'Non-Discrimination & Harassment Policy', 'Unlawful Harassment', 'No Retaliation', 'Complaint Procedure', 'Zero Tolerance', 'Availability to work', 'Time Off requests', 'Calling in procedures/emergencies', 'Tardiness', 'Cover

In [14]:
#prepare text with sections and their subsiquent text
prompt_organise_text ="please organise the employee handbook in the following sections"+ str(employee_section_list) + " find the \
subsiquent text for each section and add them in a document that has section name and the text" + full_text + " Important to Keep the sentences \
as they are from the document dont change original text and style. If a section is missing, leave it blank."

In [15]:
organised_text= model_google.generate_content(prompt_organise_text).text

In [16]:
organised_text

'## Crew Member Handbook\n\n**January, 2015**\n\n**Main Sections:**\n\n* **AT-WILL EMPLOYMENT STATEMENT**\n\nThese policies are intended to provide you with general information about STC, ENCE, NAV, and JENESIS,\nInc. (also referred to as the “Company”) employment policies, practices, guidelines and benefits. They\nare not intended to create legally enforceable obligations on the part of the Company or its employees.\nThe policies herein supersede and replace all previous personnel policies, practices, and guidelines that\nare inconsistent with any of the provisions contained herein. The Company reserves full discretion to add\nto, modify or delete provision of this Employee Handbook at any time and without advance notice. Any\nwords or phrases used in this Employee handbook shall be interpreted by the Company in its sole and\nexclusive discretion.\n\nTHIS HANDBOOK IS NOT INTENDED TO CREATE EITHER AN EXPRESS OR AN IMPLIED CONTRACT OF\nEMPLOYMENT. RATHER, IT IS INTENDED TO DESCRIBE THE 

In [17]:
import json
import re

# List of sections from the Employee Handbook
sections = cleaned_sections  # Assuming this is the variable holding your section names
# Initialize the dictionary for JSON structure
data = {"file_name": "EmployeeHandbook", "content": {}}

# Load full text from PDF extraction (use your variable containing the document text)
full_text = organised_text  # Replace with the actual full text string

# Iterate through each section to extract content
for i, section in enumerate(cleaned_sections):
    # Create regex pattern to find each section's content
    if i < len(cleaned_sections) - 1:
        # Capture content up to the next section header
        next_section = cleaned_sections[i + 1]
        pattern = rf"{re.escape(section)}(.*?)(?={re.escape(next_section)})"
    else:
        # For the last section, capture everything until the end of the document
        pattern = rf"{re.escape(section)}(.*)"

    # Search for the content under the current section header
    match = re.search(pattern, full_text, re.DOTALL)
    if match:
        # Clean and assign the matched text to the corresponding section in JSON
        data["content"][section] = match.group(1).strip()

# Save the JSON file
with open("employee_handbook.json", "w") as f:
    json.dump(data, f, indent=2)

print("JSON file created successfully!")



JSON file created successfully!


In [18]:
data

{'file_name': 'EmployeeHandbook',
 'content': {'Main Sections:': '**\n\n* **',
  'AT-WILL EMPLOYMENT STATEMENT': '**\n\nThese policies are intended to provide you with general information about STC, ENCE, NAV, and JENESIS,\nInc. (also referred to as the “Company”) employment policies, practices, guidelines and benefits. They\nare not intended to create legally enforceable obligations on the part of the Company or its employees.\nThe policies herein supersede and replace all previous personnel policies, practices, and guidelines that\nare inconsistent with any of the provisions contained herein. The Company reserves full discretion to add\nto, modify or delete provision of this Employee Handbook at any time and without advance notice. Any\nwords or phrases used in this Employee handbook shall be interpreted by the Company in its sole and\nexclusive discretion.\n\nTHIS HANDBOOK IS NOT INTENDED TO CREATE EITHER AN EXPRESS OR AN IMPLIED CONTRACT OF\nEMPLOYMENT. RATHER, IT IS INTENDED TO DE

In [19]:
import json
from sentence_transformers import SentenceTransformer

# Load JSON file
with open("/content/employee_handbook.json", "r") as f:
    data = json.load(f)

# Extract section text
sections = list(data["content"].keys())
section_texts = list(data["content"].values())

In [20]:
sections

['Main Sections:',
 'AT-WILL EMPLOYMENT STATEMENT',
 'OPEN DOOR POLICY',
 'DISCIPLINARY STATEMENT',
 'GROOMING POLICY',
 'WORK SCHEDULE PROCEDURES',
 'NON-DISCRIMINATION & HARASSMENT POLICY',
 'SECURITY PROCEDURES',
 'CASH REGISTER SHORTAGES AND PROCEDURES',
 'BREAK POLICY',
 'SMOKE POLICY',
 'EMPLOYEE MEAL POLICY',
 'WORKPLACE BEHAVIOR',
 'SPECIAL STORE RULES',
 'MISUSE OF COMPANY PRODUCT AND/OR EQUIPMENT',
 'SOLICITATION AND DISTRIBUTION POLICY',
 'CRIMINAL SEXUAL OFFENSE',
 'EEO Policy',
 'Americans With Disabilities Act',
 'Non-Discrimination & Harassment Policy',
 'Unlawful Harassment',
 'No Retaliation',
 'Complaint Procedure',
 'Availability to work',
 'Time Off requests',
 'Calling in procedures/emergencies',
 'Tardiness',
 'Covering your shift',
 'No Call, No Show Policy',
 'Work Schedule',
 'Basic Leave Entitlement',
 'Military Family Leave Entitlements',
 'Benefits and Protections']

In [21]:
section_texts

['**\n\n* **',
 '**\n\nThese policies are intended to provide you with general information about STC, ENCE, NAV, and JENESIS,\nInc. (also referred to as the “Company”) employment policies, practices, guidelines and benefits. They\nare not intended to create legally enforceable obligations on the part of the Company or its employees.\nThe policies herein supersede and replace all previous personnel policies, practices, and guidelines that\nare inconsistent with any of the provisions contained herein. The Company reserves full discretion to add\nto, modify or delete provision of this Employee Handbook at any time and without advance notice. Any\nwords or phrases used in this Employee handbook shall be interpreted by the Company in its sole and\nexclusive discretion.\n\nTHIS HANDBOOK IS NOT INTENDED TO CREATE EITHER AN EXPRESS OR AN IMPLIED CONTRACT OF\nEMPLOYMENT. RATHER, IT IS INTENDED TO DESCRIBE THE COMPANY AND ITS PRESENT POLICIES\nAND PROCEDURES. YOUR EMPLOYMENT AND COMPENSATION IS 

In [22]:
# LOAD THE EMBEDDING MODEL

In [23]:
# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Choose any model compatible with your needs

# Generate embeddings for each section
section_embeddings = model.encode(section_texts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
section_embeddings

array([[-0.05084787,  0.02643378,  0.05417842, ...,  0.04808166,
         0.04436529, -0.01833471],
       [-0.07553912,  0.10278825,  0.01103849, ..., -0.0246409 ,
         0.0384501 , -0.10899132],
       [-0.0066349 , -0.00985531,  0.08342   , ...,  0.02956251,
        -0.01961053,  0.06052883],
       ...,
       [ 0.00120846,  0.05532597, -0.0264305 , ...,  0.00278608,
         0.08066323,  0.01806183],
       [-0.03649952, -0.00623218, -0.0181979 , ..., -0.00904328,
         0.01374401,  0.03084416],
       [-0.02651832,  0.07167754, -0.00452047, ...,  0.01654497,
         0.06374805, -0.0105508 ]], dtype=float32)

In [25]:
import faiss

In [26]:
# Choose an appropriate index type (FlatL2 is a good starting point)
index = faiss.IndexFlatL2(section_embeddings.shape[1])

# Add your embeddings to the index
index.add(section_embeddings.astype('float32'))

# Save the index to a file
faiss.write_index(index, "faiss_index.bin")

In [27]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7a999440ae80> >

In [28]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [29]:
sections

['Main Sections:',
 'AT-WILL EMPLOYMENT STATEMENT',
 'OPEN DOOR POLICY',
 'DISCIPLINARY STATEMENT',
 'GROOMING POLICY',
 'WORK SCHEDULE PROCEDURES',
 'NON-DISCRIMINATION & HARASSMENT POLICY',
 'SECURITY PROCEDURES',
 'CASH REGISTER SHORTAGES AND PROCEDURES',
 'BREAK POLICY',
 'SMOKE POLICY',
 'EMPLOYEE MEAL POLICY',
 'WORKPLACE BEHAVIOR',
 'SPECIAL STORE RULES',
 'MISUSE OF COMPANY PRODUCT AND/OR EQUIPMENT',
 'SOLICITATION AND DISTRIBUTION POLICY',
 'CRIMINAL SEXUAL OFFENSE',
 'EEO Policy',
 'Americans With Disabilities Act',
 'Non-Discrimination & Harassment Policy',
 'Unlawful Harassment',
 'No Retaliation',
 'Complaint Procedure',
 'Availability to work',
 'Time Off requests',
 'Calling in procedures/emergencies',
 'Tardiness',
 'Covering your shift',
 'No Call, No Show Policy',
 'Work Schedule',
 'Basic Leave Entitlement',
 'Military Family Leave Entitlements',
 'Benefits and Protections']

**<h1>Start Retriving Q & A of handbook</h1>**

In [32]:
import numpy as np

# Function to embed text using your chosen embedding model
def embed_text(text):
    # Implement your embedding logic here
    embedding = embedding_model.encode(text)  # Replace with actual embedding logic
    return np.array(embedding)  # Ensure it's a numpy array

# Function to retrieve similar sections based on a query
def retrieve_similar_sections(query, index, your_documents, k):
    # Embed the query
    query_embedding = embed_text(query)
    print("Shape of Query Embedding:", query_embedding.shape)

    # Perform a search on the FAISS index
    distances, indices = index.search(query_embedding.reshape(1, -1), k)

    # Handle empty results
    if len(indices[0]) == 0:
        return ["No relevant sections found."]

    # Retrieve matched sections
    matched_sections = [your_documents[idx] for idx in indices[0] if idx < len(your_documents)]
    print(matched_sections,"Sections found")
    return matched_sections

# Function to generate a response using the Google Gen AI model
def generate_response(query, index, your_documents):
    # Retrieve the sections
    matched_sections = retrieve_similar_sections(query, index, your_documents, k=1)

    context = "\n\n".join(matched_sections)
    combined_input = f" This is the question from an employee : {query} and refer to the \n\nContext:\n{context} and answer \
    the question in a professional and polite in detail using the context and using language in the context "
    print("Input to model: ", combined_input)

    # Generate response using the Google Gen AI model
    response = model_google.generate_content(combined_input)
    return response

# Example usage
query_question ="Can I smoke in the break room what is the policy? "
query = query_question  # Example query
response = generate_response(query, index, section_texts)
print("\n\n THIS IS THE ANSWER TO YOUR QUESTION","\n")
print(response.text)

Shape of Query Embedding: (384,)
['**\n\nAll of our restaurants are smoke free. In efforts to promote employee health and due to the new Michigan Smoking\nLaw, we will no longer permit smoke breaks. While on your meal break, you may smoke, chew tobacco, or use\nelectronic cigarettes in your vehicle or in the corral. Smoking, chewing tobacco, or using electronic cigarettes will not\nbe permitted anywhere else on our property. Once the lobby doors are locked, you will not be allowed to go outside\nto smoke, chew tobacco, or use electronic cigarettes in your vehicle or in the corral.\n\nElectronic cigarettes are not allowed due to food safety concerns.\n\nAnyone under the age of 18 is not permitted to smoke or have tobacco products on McDonald’s premises. This is\naccording to Michigan State law and McDonald’s policy.\n\n* **'] Sections found
Input to model:   This is the question from an employee : Can I smoke in the break room what is the policy?  and refer to the 

Context:
**

All of 

In [33]:
# Example 1 USAGE EMPLOYEE QUESTION
query_question ="How many holidays can I take ? "
query = query_question  # Example query
response = generate_response(query, index, section_texts)
print("\n\nTHIS IS THE ANSWER TO YOUR QUERY","\n")
print(response.text)

Shape of Query Embedding: (384,)
['Eligible employees whose spouse, son, daughter or parent is on covered active duty or call to\ncovered active duty status may use their 12-week leave entitlement to address certain qualifying\nexigencies. Qualifying exigencies may include attending certain military events, arranging for\nalternative childcare, addressing certain financial and legal arrangements, attending certain\ncounseling sessions, and attending post-deployment reintegration briefings.\n\nFMLA also includes a special leave entitlement that permits eligible employees to take up to 26\nweeks of leave to care for a covered service- member during a single 12-month period. A\ncovered service member is:\n(1) a current member of the Armed Forces, including a member of the National Guard or\nReserves, who is undergoing medical treatment, recuperation or therapy, is otherwise in\noutpatient status, or is otherwise on the temporary disability retired list, for a serious injury or\nillness*; 

In [34]:
# Example 2 USAGE EMPLOYEE QUESTION
query_question ="What are my medical benefits"
query = query_question  # Example query
response = generate_response(query, index, section_texts)
print("\n\nTHIS IS THE ANSWER TO YOUR QUERY","\n")
print(response.text)

Shape of Query Embedding: (384,)
['During FMLA leave, the employer must maintain the employee’s health coverage under any\n“group health plan” on the same terms as if the employee had continued to work. Upon return\nfrom FMLA leave, most employees must be restored to their original or equivalent positions\nwith equivalent pay, benefits, and other employment terms.\n\nUse of FMLA leave cannot result in the loss of any employment benefit that accrued prior to the\nstart of an employee’s leave.'] Sections found
Input to model:   This is the question from an employee : What are my medical benefits and refer to the 

Context:
During FMLA leave, the employer must maintain the employee’s health coverage under any
“group health plan” on the same terms as if the employee had continued to work. Upon return
from FMLA leave, most employees must be restored to their original or equivalent positions
with equivalent pay, benefits, and other employment terms.

Use of FMLA leave cannot result in the lo

In [35]:
# Example 3 USAGE EMPLOYEE QUESTION
query_question ="How many sick leaves I get"
query = query_question  # Example query
response = generate_response(query, index, section_texts)
print("\n\nTHIS IS THE ANSWER TO YOUR QUERY","\n")
print(response.text)

Shape of Query Embedding: (384,)
['If you are ill and will not be able to work, we ask that you call and speak to a Manager at least 2 hours in advance\nof the start of your shift. This must be done verbally – A TEXT MESSAGE IS NOT ACCEPTABLE. If it is an early\nmorning shift, we ask that you call the evening before and then in the morning again. This ensures that the\nmessage is delivered. Calling in without a 2 hour notice will result in written documentation. Calling in sick when\nyou are not sick or for an emergency when there is none is a terminable offence. If calling off is a continuous\nproblem, there will be written documentation and discipline will result. All employees who are off work with a\ndoctor’s note must have a doctor’s note stating they are able to return to work. This also applies to maternity leaves.\nRequests for leave of absence must be in writing. See your General Manager for the correct form\n\n4.'] Sections found
Input to model:   This is the question from an

In [36]:
# Example 3 USAGE EMPLOYEE QUESTION
query_question ="When is my Salary Paid"
query = query_question  # Example query
response = generate_response(query, index, section_texts)
print("\n\nTHIS IS THE ANSWER TO YOUR QUERY","\n")
print(response.text)

Shape of Query Embedding: (384,)
['Your work schedule will be set up to meet your needs, other employees’ needs, and the restaurant’s needs. If your\navailability should change, please notify us in writing at least 2 weeks in advance of the change. Should your\navailability change, your rate of pay MAY degrease or increase accordingly. Your schedule will not change until the\nproper form is completed and signed. New employees cannot change their availability for 6 months unless there is a\nspecific circumstance that must be approved in writing by the Area Supervisor.\n\n2.'] Sections found
Input to model:   This is the question from an employee : When is my Salary Paid and refer to the 

Context:
Your work schedule will be set up to meet your needs, other employees’ needs, and the restaurant’s needs. If your
availability should change, please notify us in writing at least 2 weeks in advance of the change. Should your
availability change, your rate of pay MAY degrease or increase accord